In [1]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Masking, TimeDistributed, Activation, Bidirectional, Lambda, Dropout
from sklearn.model_selection import train_test_split

In [2]:
# Download and unzip dataset
import gdown

url = 'https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu'
output = 'all_wav.zip'
gdown.download(url, output, quiet=False)
!unzip -q 'all_wav.zip' -d '/content/all_wav'

url = 'https://drive.google.com/uc?id=1vqvn0F0YYhEFbzLgP9wJ36vyInUnO5b5'
output = 'dataset.csv'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu
From (redirected): https://drive.google.com/uc?id=1jyvhdZHn0s5Owkr21k5Ff-c96sIQLtEu&confirm=t&uuid=a5948018-46fc-4784-9157-8452ad93c4b4
To: /content/all_wav.zip
100%|██████████| 2.48G/2.48G [00:25<00:00, 96.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vqvn0F0YYhEFbzLgP9wJ36vyInUnO5b5
To: /content/dataset.csv
100%|██████████| 2.87M/2.87M [00:00<00:00, 18.1MB/s]


'dataset.csv'

In [3]:
# Load dataset
df = pd.read_csv('/content/dataset.csv')
df.head();

In [4]:
# Update paths for WAV files
df['wav_filename'] = df['wav_filename'].apply(lambda x: x.replace('./all_wav/', '/content/all_wav/all_wav/'))

# Filter dataset to include only existing WAV files
df = df[df['wav_filename'].apply(os.path.isfile)]

# Character map for Persian characters
char_map_str = """
' 0
<SPACE> 1
ا 2
ب 3
پ 4
ت 5
ث 6
ج 7
چ 8
ح 9
خ 10
د 11
ذ 12
ر 13
ز 14
ژ 15
س 16
ش 17
ص 18
ض 19
ط 20
ظ 21
ع 22
غ 23
ف 24
ق 25
ک 26
گ 27
ل 28
م 29
ن 30
و 31
ه 32
ی 33
، 34
؟ 35
"""
char_map = {}
index_map = {}
for line in char_map_str.strip().split('\n'):
    ch, index = line.split()
    char_map[ch] = int(index)
    index_map[int(index)] = ch
index_map[1] = ' '

# Ensure space character is in char_map
char_map[' '] = char_map['<SPACE>']

# Audio processing functions
def load_audio(file_path, sr=16000):
    audio, _ = librosa.load(file_path, sr=sr)
    return audio

def extract_features(audio, n_mfcc=13, sr=16000):
    mfcc_features = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
    return mfcc_features.T

# Prepare the dataset
X = []
y = []
input_lengths = []
label_lengths = []

for index, row in df.iterrows():
    audio_path = row['wav_filename']
    audio = load_audio(audio_path)
    features = extract_features(audio)
    X.append(features)
    input_lengths.append(features.shape[0])
    label = [char_map.get(c, char_map[' ']) for c in row['transcript']]
    y.append(label)
    label_lengths.append(len(label))

if len(X) == 0 or len(y) == 0:
    raise ValueError("No valid audio files were found. Please check the dataset and the paths.")

X = tf.keras.preprocessing.sequence.pad_sequences(X, padding='post', dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding='post', value=-1)

# Define the model
input_data = Input(name='the_input', shape=(None, 13))
masking_layer = Masking(mask_value=0.0)(input_data)
bilstm_layer_1 = Bidirectional(LSTM(128, return_sequences=True))(masking_layer)
dropout_1 = Dropout(0.5)(bilstm_layer_1)
bilstm_layer_2 = Bidirectional(LSTM(128, return_sequences=True))(dropout_1)
dropout_2 = Dropout(0.5)(bilstm_layer_2)
time_dense = TimeDistributed(Dense(len(char_map) + 1))(dropout_2)
y_pred = Activation('softmax', name='activation')(time_dense)

# Define the CTC loss function
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
    return tf.keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)

# Compile the model with CTC loss
labels = Input(name='the_labels', shape=[None], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([y_pred, labels, input_length, label_length])

model = Model(inputs=[input_data, labels, input_length, label_length], outputs=loss_out)
model.compile(optimizer=tf.keras.optimizers.Adam(), loss={'ctc': lambda y_true, y_pred: y_pred})

# Split the data into training and validation sets
X_train, X_val, y_train, y_val, input_length_train, input_length_val, label_length_train, label_length_val = train_test_split(X, y, input_lengths, label_lengths, test_size=0.2, random_state=42)

# Data generator
def data_generator(X, y, input_lengths, label_lengths, batch_size=32):
    while True:
        for i in range(0, len(X), batch_size):
            X_batch = X[i:i+batch_size]
            y_batch = y[i:i+batch_size]
            input_lengths_batch = input_lengths[i:i+batch_size]
            label_lengths_batch = label_lengths[i:i+batch_size]
            yield (
                {
                    'the_input': np.array(X_batch),
                    'the_labels': np.array(y_batch),
                    'input_length': np.array(input_lengths_batch),
                    'label_length': np.array(label_lengths_batch)
                },
                {'ctc': np.zeros([len(X_batch)])}
            )

train_gen = data_generator(X_train, y_train, input_length_train, label_length_train, batch_size=32)
val_gen = data_generator(X_val, y_val, input_length_val, label_length_val, batch_size=32)

steps_per_epoch = len(X_train) // 32
validation_steps = len(X_val) // 32

# Train the model
model.fit(train_gen, steps_per_epoch=steps_per_epoch, epochs=20, validation_data=val_gen, validation_steps=validation_steps)

# Save the model
model.save('/content/asr_model.h5')

Epoch 1/20
609/609 [==============================] - 106s 141ms/step - loss: 96.8155 - val_loss: 87.9145
Epoch 2/20
609/609 [==============================] - 82s 135ms/step - loss: 86.4931 - val_loss: 79.8096
Epoch 3/20
609/609 [==============================] - 80s 131ms/step - loss: 82.0922 - val_loss: 75.6874
Epoch 4/20
609/609 [==============================] - 90s 148ms/step - loss: 79.2499 - val_loss: 73.0871
Epoch 5/20
609/609 [==============================] - 89s 146ms/step - loss: 77.2193 - val_loss: 71.2513
Epoch 6/20
609/609 [==============================] - 90s 147ms/step - loss: 75.5246 - val_loss: 69.7618
Epoch 7/20
609/609 [==============================] - 80s 130ms/step - loss: 74.2605 - val_loss: 68.4771
Epoch 8/20
609/609 [==============================] - 81s 133ms/step - loss: 73.0960 - val_loss: 67.8132
Epoch 9/20
609/609 [==============================] - 90s 149ms/step - loss: 72.1237 - val_loss: 66.7473
Epoch 10/20
609/609 [==============================] -

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 4s 4s/step


InvalidArgumentError: {{function_node __wrapped__CTCGreedyDecoder_device_/job:localhost/replica:0/task:0/device:CPU:0}} sequence_length is not a vector [Op:CTCGreedyDecoder] name: 

In [6]:
# Redefine the inference model
inference_model = Model(inputs=input_data, outputs=y_pred)

# Load the weights from the saved model
inference_model.load_weights('/content/asr_model.h5')

# Load a sample and predict
sample_index = 0
sample_features = X[sample_index]
sample_input_length = np.array([sample_features.shape[0]])

sample_features = np.expand_dims(sample_features, axis=0)
sample_input_length = np.array([sample_features.shape[1]], dtype=np.int32)

# Predict
preds = inference_model.predict(sample_features)
decoded_pred = tf.keras.backend.ctc_decode(preds, input_length=sample_input_length)[0][0]
decoded_pred = tf.keras.backend.get_value(decoded_pred)

# Ensure decoded_pred is a 1D array
decoded_pred = decoded_pred.flatten()

# Convert the decoded prediction to text
predicted_text = ''.join([index_map[i] for i in decoded_pred if i != -1])

# Actual text from the dataset
actual_text = df.iloc[sample_index]['transcript']

print(f"Predicted text: {predicted_text}")
print(f"Actual text: {actual_text}")

1/1 [==============================] - 6s 6s/step
Predicted text: تفاباتی کهدید بد
Actual text: اتفاقاتی که ندیده بودم
